In [1]:
#lets first start by importing pandas into the kernel
!pip install numpy==1.21.4
!pip install pandas==1.3.4
!pip install keras==2.1.6


In [2]:
#next we import the relevant packages
import pandas as pd 
import numpy as np 

In [3]:
#next we download and read our dataset into the pandas dataframe 
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
#now lets check how many data points we have 
concrete_data.shape

(1030, 9)

In [5]:
#therefore signifying that we only have 1000 samples to train our data on. Hence, we have to be careful to avoid overfitting the model
#now we have to check for any missing values 
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [7]:
#now we have to change the target column of our data to binary for the model to be able to compute it 
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column


In [8]:
#a quick sanity check 
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [11]:
#now to finally normalize the data by substracting the mean of the data and dividing it by its standard deviation 
predictors_norm = (predictors - predictors.mean())/predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

In [12]:
#now we import the Keras library to make our neural network
import keras 

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [13]:
#the first line of the utput from the Keras import function indicates that tensorflow was used to install the Keras Library
#now lets import the other packages that will be required
from keras.models import Sequential
from keras.layers import Dense

In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [15]:
#The above code creates a model that has one hidden layer of 10 nodes, and a ReLU activation function
#it also uses the adam optimizer and the mean squared error as the loss function 

In [16]:
#now lets import sci-kit learn to split our data 
from sklearn.model_selection import train_test_split

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [17]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [18]:
#now lets begin making our model since our data is ready by calling the below function
model = regression_model() 

In [19]:
#then we fit the model 
model.fit(X_train, y_train, epochs=50, verbose=1)



Epoch 1/50


2024-06-04 03:04:44.522892: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-06-04 03:04:44.540236: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394300000 Hz
2024-06-04 03:04:44.541061: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564bd4be0690 executing computations on platform Host. Devices:
2024-06-04 03:04:44.541109: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-06-04 03:04:44.723313: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

721/721 [==============================] - 1s 834us/step - loss: 41733.6933
Epoch 2/50
721/721 [==============================] - 0s 221us/step - loss: 11796.4102
Epoch 3/50
721/721 [==============================] - 0s 196us/step - loss: 4473.4380
Epoch 4/50
721/721 [==============================] - 0s 197us/step - loss: 3788.3752
Epoch 5/50
721/721 [==============================] - 0s 194us/step - loss: 3566.2795
Epoch 6/50
721/721 [==============================] - 0s 168us/step - loss: 3356.0118
Epoch 7/50
721/721 [==============================] - 0s 138us/step - loss: 3128.0349
Epoch 8/50
721/721 [==============================] - 0s 167us/step - loss: 2915.2622
Epoch 9/50
721/721 [==============================] - 0s 117us/step - loss: 2713.2639
Epoch 10/50
721/721 [==============================] - 0s 161us/step - loss: 2517.9639
Epoch 11/50
721/721 [==============================] - 0s 161us/step - loss: 2331.7099
Epoch 12/50
721/721 [==============================] - 0s 139

In [20]:
#next we evaluate our model on the above test data 
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 162us/step


278.6351359839578

In [21]:
#now we need to calculate the MSE between the theoretical concrete strength and the actual value
#to do this, we import the MSE function from sci-kit learn 
from sklearn.metrics import mean_squared_error

In [22]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

278.6351452648347 0.0


In [24]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Where the otal number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 27.767516855283077
MSE 2: 30.235864565210434
MSE 3: 26.63844827695186
MSE 4: 28.969983980493637
MSE 5: 28.234937711055224
MSE 6: 30.985484552229106
MSE 7: 34.13917148923411
MSE 8: 25.02054311388133
MSE 9: 29.701856174901078
MSE 10: 26.529603556907677
MSE 11: 26.48659983884941
MSE 12: 27.938142393785
MSE 13: 31.896832595751125
MSE 14: 35.02017871153007
MSE 15: 27.365214141055603
MSE 16: 24.201390312713325
MSE 17: 27.36960000590599
MSE 18: 26.597765635518194
MSE 19: 26.74026222599363
MSE 20: 27.616114718242756
MSE 21: 26.881931267895744
MSE 22: 27.952547974571054
MSE 23: 25.45269434043119
MSE 24: 26.270361279978335
MSE 25: 29.03881149538898
MSE 26: 31.956079952153573
MSE 27: 26.258368087817935
MSE 28: 25.00375519909905
MSE 29: 31.79163423556726
MSE 30: 29.421080259057696
MSE 31: 24.165206477094237
MSE 32: 26.5480263348922
MSE 33: 24.198678167892506
MSE 34: 26.429874000425862
MSE 35: 30.1872426869414
MSE 36: 30.916775200359258
MSE 37: 24.92555283111276
MSE 38: 27.45482528093949
MSE